Q1 Develop an implementation of the basic symbol-table API that uses 2-3 trees that are not necessarily balanced as the underlying data structure. Allow 3-nodes to lean either way. Hook the new node onto the bottom with a black link when inserting into a 3-node at the bottom. 

In [91]:
# https://github.com/peterhil/leftrb/blob/master/leftrb/bst.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Binary search tree.
"""

class BinarySearchTree(object):
    """
    Basic unbalanced (inefficient) binary search tree.
    For extending into different balanced binary trees.
    """
    root = None

    class Node(object):
        """
        BST tree node.
        """
        left = right = None

        def __init__(self, key, val=None):
            self.key = key
            self.val = val

        def search(self, key):
            """
            Search the subtree for a key. Return a value or None.
            """
            if self.key == key:
                return self.val if self.val is not None else self.key
            elif key < self.key and self.left:
                return self.left.search(key)
            elif self.key < key and self.right:
                return self.right.search(key)
            return None

        def insert(self, key, value=None):
            """
            Insert a node recursively.
            """
            if self.key == key:
                self.val = value
            elif key < self.key:
                if self.left is None:
                    self.left = self.__class__(key, value)
                else:
                    self.left = self.left.insert(key, value)
            else:
                if self.right is None:
                    self.right = self.__class__(key, value)
                else:
                    self.right = self.right.insert(key, value)
            return self

        def min(self):
            """
            Smallest node in the subtree.
            """
            return self.key if self.left is None else self.left.min()

        def max(self):
            """
            Largest node in the subtree.
            """
            return self.key if self.right is None else self.right.max()
    

    def search(self, key):
        """
        Search the tree with a key. Return a value or None.
        """
        return self.root.search(key) if self.root is not None else None

    def insert(self, key, value=None):
        """
        Insert a key with optional value into tree.
        """
        self.root = self.Node(key, value) if self.root is None else self.root.insert(key, value)


# https://github.com/peterhil/leftrb/blob/master/leftrb/llrb.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Leftrb/LLRB is a Left-Leaning Red-Black (LLRB) implementation
of 2–3 balanced binary search trees in Python.
This is a straightforward port of the code in the article
“Left-leaning Red-Black Trees”
http://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf
by Robert Sedgewick of Princeton University.
"""

import sys


__all__ = ['LeftRB']

RED = True
BLACK = False


def is_red(h):
    """
    Is the node (h) red?
    """
    return isinstance(h, LeftRB.Node) and h.color == RED


def is_black(h):
    """
    Is the node (h) black?
    """
    return not is_red(h)


class LeftRB(BinarySearchTree, object):
    """
    Left-Leaning Red-Black (LLRB) is an implementation of
    2–3 balanced binary search tree.
    Ported to Python from code and description on
    paper “Left-leaning Red-Black Trees” by Robert Sedgewick:
    http://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf
    """
    root = None

    class Node(BinarySearchTree.Node, object):
        """
        LeftRB tree node.
        """
        def __init__(self, key, val=None):
            super(self.__class__, self).__init__(key, val)
            self.color = RED  # new nodes are always red
            self.height = 1

        def insert(self, key, value=None):
            """
            Recursively insert a node with key and optional value into the tree below.
            """
            # Move this to the end to get 2-3 trees
            if is_red(self.left) and is_red(self.right):
                self._flip_colors()

            self = super(LeftRB.Node, self).insert(key, value)

            if is_red(self.right) and is_black(self.left):
                self = self._rotate_left()
            if is_red(self.left) and self.left and is_red(self.left.left):
                self = self._rotate_right()

            return self._setHeight()

        def size(self):
            """
            Number of nodes in the subtree below node.
            """
            # TODO Cache into self.N
            return 1 + sum(map(lambda child: child.size(), filter(None, [self.left, self.right])))

        def __repr__(self):
            return "<{0} at {1}, key={2}, value={3}, left={6}, right={7}, color={4}, height={5}>".format(
                self.__class__.__name__,
                id(self),
                self.key,
                self.val,
                'red' if is_red(self) else 'black',
                self.height,
                self.left and self.left.key or None,
                self.right and self.right.key or None,
            )

        def _fix_up(self):
            """
            Fix the Left-leaning Red-black tree properties
            with upto two rotations and a possible color flip.
            """
            if is_red(self.right):
                self = self._rotate_left()

            if is_red(self.left) and self.left and is_red(self.left.left):
                self = self._rotate_right()

            if is_red(self.left) and is_red(self.right):
                self._flip_colors()

            return self._setHeight()

        def _flip_colors(self):
            """
            Flip colors to split a 4-node
            """
            self.color = not self.color
            self.left.color = not self.left.color
            self.right.color = not self.right.color

        def _move_red_left(self):
            """
            Assuming that self is red and both self.left and self.left.left
            are black, make self.left or one of its children red.
            """
            self._flip_colors()
            if self.right and is_red(self.right.left):
                self.right = self.right._rotate_right()
                self = self._rotate_left()
                self._flip_colors()
            return self

        def _move_red_right(self):
            """
            Assuming that self is red and both self.right and self.right.left
            are black, make self.right or one of its children red.
            """
            self._flip_colors()
            if self.left and is_red(self.left.left):
                self = self._rotate_right()
                self._flip_colors()
            return self

        def _rotate_left(self):
            """
            Left rotate (right link of self)
                   V         |          V <--left or right, red or black
                   |         |          |
            out<--(x)   <<< LEFT       (s) <--in
                 // \        |         / \\  <--red
               (s)   3       |        1   (x)
               / \           |            / \
              1   2          |           2   3
            """
            x       = self.right
            self.right = x.left
            x.left  = self
            x.color = self.color
            self.color = RED
            return x

        def _rotate_right(self):
            """
            Right rotate (left link of self)
                   V         |          V <--left or right, red or black
                   |         |          |
            in--> (s)     RIGHT >>>    (x)-->out
                 // \        |         / \\  <--red
               (x)   3       |        1   (s)
               / \           |            / \
              1   2          |           2   3
            """
            x       = self.left
            self.left  = x.right
            x.right = self
            x.color = self.color
            self.color = RED
            return x

        def _delete(self, key):
            """
            Delete a node with the given key (recursively) from the tree below.
            """
            assert self.search(key) is not None

            if key < self.key:
                if is_black(self.left) and self.left and is_black(self.left.left):
                    self = self._move_red_left()
                self.left = self.left._delete(key)
            else:
                if is_red(self.left):
                    self = self._rotate_right()

                if key == self.key and self.right is None:
                    return None

                if is_black(self.right) and self.right and is_black(self.right.left):
                    self = self._move_red_right()

                if key == self.key:
                    self.value = self.right.search(self.right.min())
                    self.key = self.right.min()
                    self.right = self.right._delete_min()
                else:
                    self.right = self.right._delete(key)

            return self._fix_up()

        def _delete_min(self):
            """
            Delete the smallest node on the (left) subtree below
            while maintaining balance.
            """
            if self.left is None:
                return None

            if is_black(self.left) and self.left and is_black(self.left.left):
                self = self._move_red_left()

            self.left = self.left._delete_min()

            return self._fix_up()

        def _delete_max(self):
            """
            Delete the largest node on the (right) subtree below
            while maintaining balance.
            """
            if is_red(self.left):
                self = self._rotate_right()

            if self.right is None:
                return None

            if is_black(self.right) and self.right and is_black(self.right.left):
                self = self._move_red_right()

            self.right = self.right._delete_max()

            return self._fix_up()

        def _setHeight(self):
            """
            Update height.
            """
            self.height = 1 + max(self.left and self.left.height or 0,
                                  self.right and self.right.height or 0)
            return self


    def is_empty(self):
        """
        Is the tree empty?
        """
        return self.root is None

    def __contains__(self, key):
        """
        Does the tree contain key?
        """
        return self.search(key) is not None

    def __len__(self):
        """
        Number of nodes in the tree.
        """
        return 0 if self.root is None else self.root.size()

    def height(self):
        """
        Height of the tree.
        """
        return 0 if self.root is None else self.root.height

    def min(self):
        """
        Smallest node in the tree.
        """
        return None if self.root is None else self.root.min()

    def max(self):
        """
        Largest node in the tree.
        """
        return None if self.root is None else self.root.max()

    def insert(self, key, value=None):
        """
        Insert a key with optional value into the tree.
        """
        super(LeftRB, self).insert(key, value)
        self.root.color = BLACK

    def delete(self, key):
        """
        Delete a node with the given key from the tree.
        """
        if key not in self:
            sys.stderr.write("Tree does not contain key '{0}'.\n".format(key))
            return False

        if is_black(self.root.left) and is_black(self.root.right):
            self.root.color = RED

        if self.root is not None:
            self.root = self.root._delete(key)

        if not self.is_empty():
            self.root.color = BLACK

    def delete_min(self):
        """
        Delete the smallest node while maintaining balance.
        """
        self.root = self.root._delete_min()
        self.root.color = BLACK

    def delete_max(self):
        """
        Delete the largest node while maintaining balance.
        """
        self.root = self.root._delete_max()
        self.root.color = BLACK
        

del BinarySearchTree

In [118]:
class ST:
    tree = LeftRB()
    key_iterable = set()
    def put(self, key, value):
        self.tree.insert(key, value)
        self.key_iterable.add(key)
    
    def get(self, key):
        return self.tree.search(key)
    
    def delete(self, key):
        self.tree.delete(key)
        self.key_iterable.remove(key)
        
    def contains(self, key):
        if self.tree.search(key) is not None:
            return True
        else:
            return False
        
    def isEmpty(self):
        if not self.tree.root:
            return True
        else:
            return False
    
    def size(self):
        return self.tree.root.size()
        
    def keys(self):
        return self.key_iterable
        


In [121]:
def Q1_experiment():
    keys = ["S", "E", "A", "R", "C", "H", "E", "X", "A", "M", "P", "L", "E"]
    values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    symbol_table = ST()
    print("Initialized empty symbol table")
    print("isEmpty() returns {}".format(symbol_table.isEmpty()))
    print("Adding keys: {} and values: {}".format(keys, values))
    for key, value in zip(keys, values):
        symbol_table.put(key, value)
    print("isEmpty() returns {}".format(symbol_table.isEmpty()))
    print("keys() returns {}".format(symbol_table.keys()))
    print("size() returns {}".format(symbol_table.size()))
    print("contains(S) returns {}".format(symbol_table.contains("S")))
    print("delete(S)".format(symbol_table.delete("S")))
    print("keys() returns {}".format(symbol_table.keys()))
    print("size() returns {}".format(symbol_table.size()))
    print("contains(S) returns {}".format(symbol_table.contains("S")))


    
Q1_experiment()

Initialized empty symbol table
isEmpty() returns False
Adding keys: ['S', 'E', 'A', 'R', 'C', 'H', 'E', 'X', 'A', 'M', 'P', 'L', 'E'] and values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
isEmpty() returns False
keys() returns {'R', 'S', 'X', 'H', 'P', 'L', 'A', 'M', 'E', 'C'}
size() returns 10
contains(S) returns True
delete(S)
keys() returns {'R', 'X', 'H', 'P', 'L', 'A', 'M', 'E', 'C'}
size() returns 9
contains(S) returns False


Q2. Run experiments to develop a hypothesis estimating the average path length in a tree built from (i) N-random insertions. (ii) N-sorted insertions? 

In [127]:
# https://github.com/peterhil/leftrb/blob/master/leftrb/bst.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Binary search tree.
"""

class BinarySearchTree(object):
    """
    Basic unbalanced (inefficient) binary search tree.
    For extending into different balanced binary trees.
    """
    root = None

    class Node(object):
        """
        BST tree node.
        """
        left = right = None

        def __init__(self, key, val=None):
            self.key = key
            self.val = val

        def search(self, key):
            """
            Search the subtree for a key. Return a value or None.
            """
            if self.key == key:
                return self.val if self.val is not None else self.key
            elif key < self.key and self.left:
                return self.left.search(key)
            elif self.key < key and self.right:
                return self.right.search(key)
            return None

        def insert(self, key, value=None):
            """
            Insert a node recursively.
            """
            if self.key == key:
                self.val = value
            elif key < self.key:
                if self.left is None:
                    self.left = self.__class__(key, value)
                else:
                    self.left = self.left.insert(key, value)
            else:
                if self.right is None:
                    self.right = self.__class__(key, value)
                else:
                    self.right = self.right.insert(key, value)
            return self

        def min(self):
            """
            Smallest node in the subtree.
            """
            return self.key if self.left is None else self.left.min()

        def max(self):
            """
            Largest node in the subtree.
            """
            return self.key if self.right is None else self.right.max()
    

    def search(self, key):
        """
        Search the tree with a key. Return a value or None.
        """
        return self.root.search(key) if self.root is not None else None

    def insert(self, key, value=None):
        """
        Insert a key with optional value into tree.
        """
        self.root = self.Node(key, value) if self.root is None else self.root.insert(key, value)

# https://stackoverflow.com/questions/36242046/internal-path-length-of-red-black-tree

def internal_path_length(root_node, curr_depth = 0):
        """
        Computes internal path length
        """
        if not root_node:
            return 0
        else:
            return curr_depth + internal_path_length(root_node.left, curr_depth+1) + internal_path_length(root_node.right, curr_depth+1)
        

In [134]:
from random import shuffle

def Q2_experiment():
    num_trials = 1000
    shuffle_results = []
    sorted_results = []
    for N in range(10, 950, 10):
        shuffle_mean = []
        sorted_mean = []
        for _ in range(num_trials):
            shuffled_inserts = list(range(N))
            sorted_inserts = shuffled_inserts.copy()
            shuffle(shuffled_inserts)

            shuffle_experiment = BinarySearchTree()
            sorted_experiment = BinarySearchTree()

            for insert in shuffled_inserts:
                shuffle_experiment.insert(insert, insert)
            for insert in sorted_inserts:
                sorted_experiment.insert(insert, insert)

            shuffle_internal_path_length = internal_path_length(shuffle_experiment.root)
            shuffle_mean.append(shuffle_internal_path_length / N)

            sorted_internal_path_length = internal_path_length(sorted_experiment.root)
            sorted_mean.append(sorted_internal_path_length / N)
        print("After {} trials, the average path length for {} random insertions is {}".format(num_trials, N, sum(shuffle_mean)/len(shuffle_mean)))
        print("After {} trials, the average path length for {} sorted insertions is {}".format(num_trials, N, sum(sorted_mean)/len(sorted_mean)))

        shuffle_results.append(sum(shuffle_mean)/len(shuffle_mean))
        sorted_results.append(sum(sorted_mean)/len(sorted_mean))
        
Q2_experiment()


After 1000 trials, the average path length for 10 random insertions is 2.442899999999998
After 1000 trials, the average path length for 10 sorted insertions is 4.5
After 1000 trials, the average path length for 20 random insertions is 3.5681999999999987
After 1000 trials, the average path length for 20 sorted insertions is 9.5
After 1000 trials, the average path length for 30 random insertions is 4.240733333333332
After 1000 trials, the average path length for 30 sorted insertions is 14.5
After 1000 trials, the average path length for 40 random insertions is 4.786549999999995
After 1000 trials, the average path length for 40 sorted insertions is 19.5
After 1000 trials, the average path length for 50 random insertions is 5.192620000000009
After 1000 trials, the average path length for 50 sorted insertions is 24.5
After 1000 trials, the average path length for 60 random insertions is 5.527400000000006
After 1000 trials, the average path length for 60 sorted insertions is 29.5
After 1000 

KeyboardInterrupt: 

Q3. Write a program that computes the percentage of red nodes in a given red-black tree. Test program by running at least 100 trials of the experiment of increasing N random keys into an initially empty tree for N=10^4, 10^5 and 10^6 and formulate a hypothesis.

In [68]:
# https://github.com/peterhil/leftrb/blob/master/leftrb/bst.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Binary search tree.
"""

class BinarySearchTree(object):
    """
    Basic unbalanced (inefficient) binary search tree.
    For extending into different balanced binary trees.
    """
    root = None

    class Node(object):
        """
        BST tree node.
        """
        left = right = None

        def __init__(self, key, val=None):
            self.key = key
            self.val = val

        def search(self, key):
            """
            Search the subtree for a key. Return a value or None.
            """
            if self.key == key:
                return self.val if self.val is not None else self.key
            elif key < self.key and self.left:
                return self.left.search(key)
            elif self.key < key and self.right:
                return self.right.search(key)
            return None

        def insert(self, key, value=None):
            """
            Insert a node recursively.
            """
            if self.key == key:
                self.val = value
            elif key < self.key:
                if self.left is None:
                    self.left = self.__class__(key, value)
                else:
                    self.left = self.left.insert(key, value)
            else:
                if self.right is None:
                    self.right = self.__class__(key, value)
                else:
                    self.right = self.right.insert(key, value)
            return self

        def min(self):
            """
            Smallest node in the subtree.
            """
            return self.key if self.left is None else self.left.min()

        def max(self):
            """
            Largest node in the subtree.
            """
            return self.key if self.right is None else self.right.max()
    

    def search(self, key):
        """
        Search the tree with a key. Return a value or None.
        """
        return self.root.search(key) if self.root is not None else None

    def insert(self, key, value=None):
        """
        Insert a key with optional value into tree.
        """
        self.root = self.Node(key, value) if self.root is None else self.root.insert(key, value)


# https://github.com/peterhil/leftrb/blob/master/leftrb/llrb.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Leftrb/LLRB is a Left-Leaning Red-Black (LLRB) implementation
of 2–3 balanced binary search trees in Python.
This is a straightforward port of the code in the article
“Left-leaning Red-Black Trees”
http://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf
by Robert Sedgewick of Princeton University.
"""

import sys


__all__ = ['LeftRB']

RED = True
BLACK = False


def is_red(h):
    """
    Is the node (h) red?
    """
    return isinstance(h, LeftRB.Node) and h.color == RED


def is_black(h):
    """
    Is the node (h) black?
    """
    return not is_red(h)


class LeftRB(BinarySearchTree, object):
    """
    Left-Leaning Red-Black (LLRB) is an implementation of
    2–3 balanced binary search tree.
    Ported to Python from code and description on
    paper “Left-leaning Red-Black Trees” by Robert Sedgewick:
    http://www.cs.princeton.edu/~rs/talks/LLRB/LLRB.pdf
    """
    root = None

    class Node(BinarySearchTree.Node, object):
        """
        LeftRB tree node.
        """
        def __init__(self, key, val=None):
            super(self.__class__, self).__init__(key, val)
            self.color = RED  # new nodes are always red
            self.height = 1

        def insert(self, key, value=None):
            """
            Recursively insert a node with key and optional value into the tree below.
            """
            # Move this to the end to get 2-3 trees
            if is_red(self.left) and is_red(self.right):
                self._flip_colors()

            self = super(LeftRB.Node, self).insert(key, value)

            if is_red(self.right) and is_black(self.left):
                self = self._rotate_left()
            if is_red(self.left) and self.left and is_red(self.left.left):
                self = self._rotate_right()

            return self._setHeight()

        def size(self):
            """
            Number of nodes in the subtree below node.
            """
            # TODO Cache into self.N
            return 1 + sum(map(lambda child: child.size(), filter(None, [self.left, self.right])))

        def __repr__(self):
            return "<{0} at {1}, key={2}, value={3}, left={6}, right={7}, color={4}, height={5}>".format(
                self.__class__.__name__,
                id(self),
                self.key,
                self.val,
                'red' if is_red(self) else 'black',
                self.height,
                self.left and self.left.key or None,
                self.right and self.right.key or None,
            )

        def _fix_up(self):
            """
            Fix the Left-leaning Red-black tree properties
            with upto two rotations and a possible color flip.
            """
            if is_red(self.right):
                self = self._rotate_left()

            if is_red(self.left) and self.left and is_red(self.left.left):
                self = self._rotate_right()

            if is_red(self.left) and is_red(self.right):
                self._flip_colors()

            return self._setHeight()

        def _flip_colors(self):
            """
            Flip colors to split a 4-node
            """
            self.color = not self.color
            self.left.color = not self.left.color
            self.right.color = not self.right.color

        def _move_red_left(self):
            """
            Assuming that self is red and both self.left and self.left.left
            are black, make self.left or one of its children red.
            """
            self._flip_colors()
            if self.right and is_red(self.right.left):
                self.right = self.right._rotate_right()
                self = self._rotate_left()
                self._flip_colors()
            return self

        def _move_red_right(self):
            """
            Assuming that self is red and both self.right and self.right.left
            are black, make self.right or one of its children red.
            """
            self._flip_colors()
            if self.left and is_red(self.left.left):
                self = self._rotate_right()
                self._flip_colors()
            return self

        def _rotate_left(self):
            """
            Left rotate (right link of self)
                   V         |          V <--left or right, red or black
                   |         |          |
            out<--(x)   <<< LEFT       (s) <--in
                 // \        |         / \\  <--red
               (s)   3       |        1   (x)
               / \           |            / \
              1   2          |           2   3
            """
            x       = self.right
            self.right = x.left
            x.left  = self
            x.color = self.color
            self.color = RED
            return x

        def _rotate_right(self):
            """
            Right rotate (left link of self)
                   V         |          V <--left or right, red or black
                   |         |          |
            in--> (s)     RIGHT >>>    (x)-->out
                 // \        |         / \\  <--red
               (x)   3       |        1   (s)
               / \           |            / \
              1   2          |           2   3
            """
            x       = self.left
            self.left  = x.right
            x.right = self
            x.color = self.color
            self.color = RED
            return x

        def _delete(self, key):
            """
            Delete a node with the given key (recursively) from the tree below.
            """
            assert self.search(key) is not None

            if key < self.key:
                if is_black(self.left) and self.left and is_black(self.left.left):
                    self = self._move_red_left()
                self.left = self.left._delete(key)
            else:
                if is_red(self.left):
                    self = self._rotate_right()

                if key == self.key and self.right is None:
                    return None

                if is_black(self.right) and self.right and is_black(self.right.left):
                    self = self._move_red_right()

                if key == self.key:
                    self.value = self.right.search(self.right.min())
                    self.key = self.right.min()
                    self.right = self.right._delete_min()
                else:
                    self.right = self.right._delete(key)

            return self._fix_up()

        def _delete_min(self):
            """
            Delete the smallest node on the (left) subtree below
            while maintaining balance.
            """
            if self.left is None:
                return None

            if is_black(self.left) and self.left and is_black(self.left.left):
                self = self._move_red_left()

            self.left = self.left._delete_min()

            return self._fix_up()

        def _delete_max(self):
            """
            Delete the largest node on the (right) subtree below
            while maintaining balance.
            """
            if is_red(self.left):
                self = self._rotate_right()

            if self.right is None:
                return None

            if is_black(self.right) and self.right and is_black(self.right.left):
                self = self._move_red_right()

            self.right = self.right._delete_max()

            return self._fix_up()

        def _setHeight(self):
            """
            Update height.
            """
            self.height = 1 + max(self.left and self.left.height or 0,
                                  self.right and self.right.height or 0)
            return self


    def is_empty(self):
        """
        Is the tree empty?
        """
        return self.root is None

    def __contains__(self, key):
        """
        Does the tree contain key?
        """
        return self.search(key) is not None

    def __len__(self):
        """
        Number of nodes in the tree.
        """
        return 0 if self.root is None else self.root.size()

    def height(self):
        """
        Height of the tree.
        """
        return 0 if self.root is None else self.root.height

    def min(self):
        """
        Smallest node in the tree.
        """
        return None if self.root is None else self.root.min()

    def max(self):
        """
        Largest node in the tree.
        """
        return None if self.root is None else self.root.max()

    def insert(self, key, value=None):
        """
        Insert a key with optional value into the tree.
        """
        super(LeftRB, self).insert(key, value)
        self.root.color = BLACK

    def delete(self, key):
        """
        Delete a node with the given key from the tree.
        """
        if key not in self:
            sys.stderr.write("Tree does not contain key '{0}'.\n".format(key))
            return False

        if is_black(self.root.left) and is_black(self.root.right):
            self.root.color = RED

        if self.root is not None:
            self.root = self.root._delete(key)

        if not self.is_empty():
            self.root.color = BLACK

    def delete_min(self):
        """
        Delete the smallest node while maintaining balance.
        """
        self.root = self.root._delete_min()
        self.root.color = BLACK

    def delete_max(self):
        """
        Delete the largest node while maintaining balance.
        """
        self.root = self.root._delete_max()
        self.root.color = BLACK
        

del BinarySearchTree

In [43]:
def inorder_traverse(nood):
    red_count = 0
    if not nood:
        return 0
    red_count += inorder_traverse(nood.left)
    if is_red(nood):
        red_count += 1
    red_count += inorder_traverse(nood.right)    
    return red_count

def red_percentage(llrb):
    red_count = inorder_traverse(llrb.root)
    total_count = llrb.__len__()
    return red_count/total_count
    

In [46]:
def q3_experiment():
    num_trials = 1
    for N in [10000, 100000]: #, 1000000]:
        results = []
        for _ in range(num_trials): 
            shuffled_inserts = list(range(N))
            shuffle(shuffled_inserts)
            llrb = LeftRB()
            for insert in shuffled_inserts:
                llrb.insert(insert)
            results.append(red_percentage(llrb))
        print("Average red node percentage after {} trials is {}% for {} random nodes".format(num_trials, 100*sum(results)/len(results),N))

q3_experiment()

Average red node percentage after 1 trials is 42.65% for 10000 random nodes
Average red node percentage after 1 trials is 43.076% for 100000 random nodes


Q4. Run empirical studies to compute the average and std deviation of the average length of a path to a random node (internal path length divided by tree size) in a red-black BST built by insertion of N random keys into an initially empty tree, for N from 1 to 10,000. Do at least 1,000 trials for each size.

In [69]:
# https://stackoverflow.com/questions/36242046/internal-path-length-of-red-black-tree

def internal_path_length(root_node, curr_depth = 0):
        """
        Computes internal path length
        """
        if not root_node:
            return 0
        else:
            return curr_depth + internal_path_length(root_node.left, curr_depth+1) + internal_path_length(root_node.right, curr_depth+1)

In [71]:
from statistics import stdev

def q4_experiment():
    num_trials = 1000
    for N in [1] + list(range(500,10001, 500)):
        result_vector = []
        for _ in range(num_trials): 
            shuffled_inserts = list(range(N))
            shuffle(shuffled_inserts)
            llrb = LeftRB()
            for insert in shuffled_inserts:
                llrb.insert(insert)
            result_vector.append(internal_path_length(llrb.root)/N)
            
        standard_dev = stdev(result_vector)
        mean = sum(result_vector) / len(result_vector)
        
        print("For {} trials and {} random node insertions, the average path length is {} with std dev {}".format(num_trials, N, mean, standard_dev))
                    
q4_experiment()

For 10 trials and 1 random node insertions, the average path length is 0.0 with std dev 0.0
For 10 trials and 500 random node insertions, the average path length is 7.3222000000000005 with std dev 0.07936105958074013
For 10 trials and 1000 random node insertions, the average path length is 8.320699999999999 with std dev 0.048458573372865
For 10 trials and 1500 random node insertions, the average path length is 9.010066666666665 with std dev 0.10001059203163694
For 10 trials and 2000 random node insertions, the average path length is 9.335799999999999 with std dev 0.0759243775813216
For 10 trials and 2500 random node insertions, the average path length is 9.72268 with std dev 0.056219901577051375
For 10 trials and 3000 random node insertions, the average path length is 9.99 with std dev 0.07126840243182915
For 10 trials and 3500 random node insertions, the average path length is 10.263571428571428 with std dev 0.09884202563558696
For 10 trials and 4000 random node insertions, the averag

Q5. Implement the rank() and select() ordered operations for a BST. Use data set linked below. (i) What is the value of select(7) for the data set? (ii)  What is the value of rank(7) for the data set? [10 points]

In [72]:
# https://github.com/peterhil/leftrb/blob/master/leftrb/bst.py

#!/usr/bin/env python -u
# encoding: utf-8
#
# Leftrb is a Left-Leaning Red-Black tree implementation in Python.
# Copyright (c) 2013, Peter Hillerström <peter.hillerstrom@gmail.com>
#
# This file is part of Leftrb.
#
# Leftrb is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as
# published by the Free Software Foundation, either version 3
# of the License, or (at your option) any later version.
#
# Leftrb is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
# See the GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public
# License along with Leftrb.  If not, see <http://www.gnu.org/licenses/>.
"""
Binary search tree.
"""

class BinarySearchTree(object):
    """
    Basic unbalanced (inefficient) binary search tree.
    For extending into different balanced binary trees.
    """
    root = None

    class Node(object):
        """
        BST tree node.
        """
        left = right = None

        def __init__(self, key, val=None):
            self.key = key
            self.val = val

        def search(self, key):
            """
            Search the subtree for a key. Return a value or None.
            """
            if self.key == key:
                return self.val if self.val is not None else self.key
            elif key < self.key and self.left:
                return self.left.search(key)
            elif self.key < key and self.right:
                return self.right.search(key)
            return None

        def insert(self, key, value=None):
            """
            Insert a node recursively.
            """
            if self.key == key:
                self.val = value
            elif key < self.key:
                if self.left is None:
                    self.left = self.__class__(key, value)
                else:
                    self.left = self.left.insert(key, value)
            else:
                if self.right is None:
                    self.right = self.__class__(key, value)
                else:
                    self.right = self.right.insert(key, value)
            return self

        def min(self):
            """
            Smallest node in the subtree.
            """
            return self.key if self.left is None else self.left.min()

        def max(self):
            """
            Largest node in the subtree.
            """
            return self.key if self.right is None else self.right.max()
    

    def search(self, key):
        """
        Search the tree with a key. Return a value or None.
        """
        return self.root.search(key) if self.root is not None else None

    def insert(self, key, value=None):
        """
        Insert a key with optional value into tree.
        """
        self.root = self.Node(key, value) if self.root is None else self.root.insert(key, value)
        

In [89]:
# rank in O(n) with inorder traversal
def inorder(node, key_queue):
    if not node:
        return -1
    inorder(node.left, key_queue)
    key_queue.append(node.key)
    inorder(node.right, key_queue)

def rank(root, key):
    q = []
    inorder(root, q)
    return q.index(key)

def select(root, key):
    q = []
    inorder(root, q)
    return q[key]
    
def Q5_experiment():
    nodes = [1,3,5,6,7,8,12]
    shuffle(nodes)
    bst = BinarySearchTree()
    for node in nodes:
        bst.insert(node)
    print(rank(bst.root, 3))
    print(select(bst.root, 1))
    
Q5_experiment()

1
3
